In [39]:
# ===== Reduced Order Modeling with NGSolve + pyMOR (POD-Galerkin) =====
from netgen.occ import *

import numpy as np
from ngsolve import *
from ngsolve import Mesh
from netgen.geom2d import unit_square

from pymor.algorithms.pod import pod
from pymor.vectorarrays.numpy import NumpyVectorSpace
import matplotlib.pyplot as plt
from collections import Counter


In [40]:
# ---------- Parameters ----------
L, W, H      = 3.0, 1.0, 1.0
baffle_thk   = 0.05
baffle_len   = 0.70*W
baffle_x     = [0.7, 1.5, 2.3]   # positions of the three baffles
half_model   = False
maxh         = 0.1

# stub parameters
Lin_ext  = 0.4   # inlet stub length
Lout_ext = 0.4   # outlet stub length
stub_frac = 0.5  # fraction of duct cross-section used for stub

# ---------- Main duct ----------
duct = Box(Pnt(0,0,0), Pnt(L, W, H))
duct.faces.name = "walls"

# ---------- Baffles ----------
fluid = duct
for i, x0 in enumerate(baffle_x, 1):
    x0min = x0 - 0.5*baffle_thk
    x0max = x0 + 0.5*baffle_thk
    zmin, zmax = 0, H

    if i == 2:  # middle baffle from TOP wall
        ymin, ymax = W - baffle_len, W
    else:       # side baffles from BOTTOM wall
        ymin, ymax = 0, baffle_len

    plate = Box(Pnt(x0min, ymin, zmin), Pnt(x0max, ymax, zmax))
    plate.faces.name = f"baffle{i}"
    fluid = fluid - plate

# ---------- Inlet stub ----------
stub_ymin = (1-stub_frac)/2 * W
stub_ymax = (1+stub_frac)/2 * W
stub_zmin = (1-stub_frac)/2 * H
stub_zmax = (1+stub_frac)/2 * H

stub_in = Box(Pnt(-Lin_ext, stub_ymin, stub_zmin),
              Pnt(0,        stub_ymax, stub_zmax))
stub_in.faces.name = "walls"           # stub sides are walls
stub_in.faces.Min(X).name = "inlet"    # only far end is inlet

# ---------- Outlet stub ----------
stub_out = Box(Pnt(L, stub_ymin, stub_zmin),
               Pnt(L+Lout_ext, stub_ymax, stub_zmax))
stub_out.faces.name = "walls"          # stub sides are walls
stub_out.faces.Max(X).name = "outlet"  # only far end is outlet

# ---------- Combine duct + stubs ----------
fluid = fluid + stub_in + stub_out

# ---------- Apply half-model cut if desired ----------
if half_model:
    sym_half = HalfSpace(Pnt(0, W/2, 0), Vec(0,1,0))
    fluid = fluid * sym_half

# ---------- Build OCC geometry and mesh ----------
geo = OCCGeometry(fluid)
m = geo.GenerateMesh(maxh=maxh)
mesh = Mesh(m)

print("Mesh elements:", mesh.ne)
print("Boundary names:", mesh.GetBoundaries())

# ---------- Count boundary elements ----------
counter = Counter()
for el in mesh.Elements(BND):
    counter[el.mat] += 1
print("Boundary facet counts:", counter)

# ---------- Visualize boundaries ----------
cf = mesh.BoundaryCF({
    "inlet":   10,
    "outlet":  20,
    "walls":   30,
    "baffle1": 40,
    "baffle2": 50,
    "baffle3": 60
}, default=0)

Draw(mesh)

Mesh elements: 16210
Boundary names: ('walls', 'walls', 'walls', 'walls', 'walls', 'walls', 'walls', 'walls', 'walls', 'baffle1', 'baffle2', 'baffle2', 'baffle2', 'walls', 'walls', 'walls', 'baffle3', 'baffle3', 'baffle3', 'walls', 'baffle1', 'baffle1', 'inlet', 'walls', 'walls', 'walls', 'walls', 'outlet')
Boundary facet counts: Counter({'walls': 3324, 'baffle2': 374, 'baffle1': 336, 'baffle3': 318, 'inlet': 56, 'outlet': 50})


In [41]:
from ngsolve import *
from ngsolve import x, y, z
from ngsolve.webgui import Draw

SetNumThreads(1)

# ------------ parameters ------------
nu_f      = 0.01          # viscosity
U0        = 1.0           # target peak inlet speed
omega0    = 0.3           # under-relaxation
tol       = 1e-4          # Picard stopping criterion
maxit     = 50            # allow more iterations if needed
gamma_gd  = 0.7           # grad-div penalty
use_backflow = True

# ------------ FE spaces ------------
V = VectorH1(mesh, order=1, dirichlet="walls|baffle1|baffle2|baffle3|inlet")
Q = H1(mesh, order=1)
fes = V*Q
(u, p), (v, q) = fes.TnT()

# inlet profile helper
def inlet_cf(U0):
    prof = U0 * (1 - (2*y/W-1)**2) * (1 - (2*z/H-1)**2)
    return CoefficientFunction((prof, 0, 0))

# lifting (velocity only)
def lifting(U0):
    g = GridFunction(fes)
    g.components[0].Set(inlet_cf(U0), BND, definedon=mesh.Boundaries("inlet"))
    return g

# ------------ Stokes warm-start ------------
gS = lifting(U0)  # directly use full U0

aS = BilinearForm(fes, symmetric=True)
aS += nu_f * InnerProduct(Grad(u), Grad(v)) * dx
aS += (-div(v)*p - q*div(u)) * dx
aS += 1e-12*p*q*dx
aS += gamma_gd * div(u)*div(v) * dx

fS = LinearForm(fes)
aS.Assemble(); fS.Assemble()
rhs = fS.vec.CreateVector(); rhs.data = fS.vec - aS.mat*gS.vec
sol = GridFunction(fes)
inv = aS.mat.Inverse(fes.FreeDofs(), inverse="pardiso")
sol.vec.data = gS.vec + inv*rhs

u_vel = GridFunction(V); u_vel.vec.data = sol.components[0].vec
p_prs = GridFunction(Q); p_prs.vec.data = sol.components[1].vec
print("Stokes L2(u) =", float(Integrate(Norm(u_vel), mesh)))

# ------------ Oseen iteration (direct to U0) ------------
n = specialcf.normal(3)
g = lifting(U0)
uk = GridFunction(V); uk.vec.data = u_vel.vec
omega = omega0
last_rel = None

for it in range(1, maxit+1):
    aO = BilinearForm(fes, symmetric=False)
    aO += nu_f * InnerProduct(Grad(u), Grad(v)) * dx
    aO += InnerProduct(Grad(u)*uk, v) * dx
    aO += (-div(v)*p - q*div(u)) * dx
    aO += 1e-12*p*q*dx
    aO += gamma_gd * div(u)*div(v) * dx

    if use_backflow:
        un = InnerProduct(uk, n)
        neg_un = IfPos(-un, -un, 0.0)   # penalize only backflow
        aO += 0.5*neg_un*InnerProduct(u, v) * ds(definedon=mesh.Boundaries("outlet"))

    fO = LinearForm(fes)
    aO.Assemble(); fO.Assemble()
    rhs = fO.vec.CreateVector(); rhs.data = fO.vec - aO.mat*g.vec
    sol = GridFunction(fes)
    inv = aO.mat.Inverse(fes.FreeDofs(), inverse="pardiso")
    sol.vec.data = g.vec + inv*rhs

    du  = GridFunction(V); du.vec.data = sol.components[0].vec - u_vel.vec
    rel = Norm(du.vec) / max(Norm(sol.components[0].vec), 1e-30)
    print(f"  iter {it:2d}: rel = {rel:.3e}, omega={omega:.2f}")

    # under-relaxed update
    u_vel.vec.data = (1-omega)*u_vel.vec + omega*sol.components[0].vec
    p_prs.vec.data = (1-omega)*p_prs.vec + omega*sol.components[1].vec
    uk.vec.data    = u_vel.vec

    if rel < tol:
        print("  converged.")
        break

    if last_rel is not None and rel > 1.25*last_rel and omega > 0.18:
        omega *= 0.7
    last_rel = rel

# ------------ Rescale velocity to U0 ------------
inlet_area = Integrate(1, mesh, BND, definedon=mesh.Boundaries("inlet"))
avg_inlet  = Integrate(Norm(u_vel), mesh, BND, definedon=mesh.Boundaries("inlet")) / inlet_area
scale = U0 / avg_inlet
print(f"\nScaling velocity field by factor {scale}")
u_vel.vec.data *= scale

avg_inlet_new = Integrate(Norm(u_vel), mesh, BND, definedon=mesh.Boundaries("inlet")) / inlet_area
print("new avg |u| on inlet =", avg_inlet_new)

# convection field for scalar problem
b = CoefficientFunction((u_vel[0], u_vel[1], u_vel[2]))
VTKOutput(ma=mesh, coefs=[u_vel], names=["u"], filename="solution_navier", subdivision=2).Do()


Stokes L2(u) = 1.2110488780460822
  iter  1: rel = 2.730e-01, omega=0.30
  iter  2: rel = 2.068e-01, omega=0.30
  iter  3: rel = 1.566e-01, omega=0.30
  iter  4: rel = 1.188e-01, omega=0.30
  iter  5: rel = 9.061e-02, omega=0.30
  iter  6: rel = 6.961e-02, omega=0.30
  iter  7: rel = 5.383e-02, omega=0.30
  iter  8: rel = 4.183e-02, omega=0.30
  iter  9: rel = 3.257e-02, omega=0.30
  iter 10: rel = 2.536e-02, omega=0.30
  iter 11: rel = 1.973e-02, omega=0.30
  iter 12: rel = 1.535e-02, omega=0.30
  iter 13: rel = 1.198e-02, omega=0.30
  iter 14: rel = 9.397e-03, omega=0.30
  iter 15: rel = 7.441e-03, omega=0.30
  iter 16: rel = 5.956e-03, omega=0.30
  iter 17: rel = 4.819e-03, omega=0.30
  iter 18: rel = 3.930e-03, omega=0.30
  iter 19: rel = 3.219e-03, omega=0.30
  iter 20: rel = 2.636e-03, omega=0.30
  iter 21: rel = 2.150e-03, omega=0.30
  iter 22: rel = 1.743e-03, omega=0.30
  iter 23: rel = 1.401e-03, omega=0.30
  iter 24: rel = 1.116e-03, omega=0.30
  iter 25: rel = 8.805e-04, om

'solution_navier'

In [43]:
# ---- Advection–diffusion PDE from Quarteroni §3.8 (Eq. 3.88) ----
#    -nu Δu + b·∇u = 0
#    u = gp on baffles, u=0 on walls+inlet,
#    Neumann (do-nothing) at outlet

from ngsolve import *
from ngsolve.webgui import Draw
from ngsolve import x, y, z


# ---- parameters ----
g1, g2, g3 = 6.0, 9.0, 12.0    # Dirichlet values on baffle1-3
Pe         = 500.0              # Peclet number (mu4)
nu         = 1.0/Pe            # diffusion coefficient (since V=L=1)

print("approx Pe_K ~", ((1.0*maxh)/(2*nu)))   # if max |b|≈1


# ---- FE space ----
# Dirichlet on walls, inlet, and baffles
V = H1(mesh, order=1,dirichlet="walls|inlet|baffle1|baffle2|baffle3")
u, v = V.TnT()


# ---- convection field b(x) ----
# u_vel from previous block
b = CoefficientFunction((u_vel[0], u_vel[1], u_vel[2]))

# ---- bilinear and linear forms ----
a = BilinearForm(V, symmetric=False)
a += nu * InnerProduct(grad(u), grad(v)) * dx    # diffusion
a += InnerProduct(b,grad(u)) * v * dx            # advection

# optional streamline-diffusion stabilization (SUPG)
hK = specialcf.mesh_size   # local element size coefficient function
tau = 1.0 * hK
a += tau * InnerProduct(b,grad(u)) * InnerProduct(b,grad(v)) * dx

f = LinearForm(V)    # no volumetric source
f.Assemble()

# ---- lifting for Dirichlet BCs: set ALL in one shot (boundary-wise CF) ----
# map boundary-name -> value
bc_vals = {
    "walls":   0.0,    # book: u=0 on walls
    "inlet":   0.0,    # book: u=0 on inlet
    "baffle1": g1,
    "baffle2": g2,
    "baffle3": g3,
    "outlet":  None    # Neumann (natural); no Dirichlet here
}

bnd_names = list(mesh.GetBoundaries())
vals_list = []
for name in bnd_names:
    v = bc_vals.get(name, None)
    vals_list.append(0.0 if v is None else float(v))
cf_bndwise = CoefficientFunction(vals_list)
g = GridFunction(V)
g.Set(cf_bndwise, BND)   # one call, boundary-wise assignment

# sanity check: averages on each baffle (should equal g1,g2,g3)
for name, target in [("baffle1", g1), ("baffle2", g2), ("baffle3", g3)]:
    area = Integrate(1, mesh, BND, definedon=mesh.Boundaries(name))
    avg  = Integrate(g, mesh, BND, definedon=mesh.Boundaries(name)) / area
    print(f"avg g on {name} = {avg} (target {target})")

# ---- assemble and solve ----
a.Assemble()
rhs = f.vec.CreateVector()
rhs.data = f.vec - a.mat * g.vec

uN = GridFunction(V)
uN.vec.data = g.vec    # start with Dirichlet lifting
inv = a.mat.Inverse(V.FreeDofs(), inverse="pardiso")
uN.vec.data += inv * rhs


vals = np.array(uN.vec)
print("min u =", vals.min())
print("max u =", vals.max())

# ---- visualize ----
print(mesh.GetBoundaries())

# Draw(uN, mesh, name="u")

from ngsolve import VTKOutput
VTKOutput(ma=mesh, coefs=[uN], names=["u"], filename="solution", subdivision=2).Do()


approx Pe_K ~ 25.0
avg g on baffle1 = 5.557620945330799 (target 6.0)
avg g on baffle2 = 8.389767345212372 (target 9.0)
avg g on baffle3 = 11.072171856016174 (target 12.0)
min u = -0.4664272909548898
max u = 12.000000000000002
('walls', 'walls', 'walls', 'walls', 'walls', 'walls', 'walls', 'walls', 'walls', 'baffle1', 'baffle2', 'baffle2', 'baffle2', 'walls', 'walls', 'walls', 'baffle3', 'baffle3', 'baffle3', 'walls', 'baffle1', 'baffle1', 'inlet', 'walls', 'walls', 'walls', 'walls', 'outlet')


'solution'